# loading

In [1]:
# library loading
import pandas as pd
import numpy as np
import re
import shutil
import os.path

In [3]:
# data loading
data = pd.read_excel("C://info_data//new_book_08_08.xlsx") # 수집 원본 데이터

data['set_isbn'] = data['set_isbn'].astype(str)
data['isbn'] = data['isbn'].astype(str)

In [8]:
data.columns

Index(['main_title', 'sub_title', 'cate', 'tot_page', 'author', 'publish',
       'date', 'size', 'wei', 'img', 'book_info', 'ref_info', 'book_review',
       'ref_review', 'index', 'ref_index', 'set_isbn', 'isbn', 'price',
       'book_type', 'ver', 'last_ver', 'pn_isbn', 'keyword', 'cate_kb'],
      dtype='object')

In [30]:
codes = data['set_isbn']
code = data['isbn']
tit = data['main_title']
rev = data['book_review']
info = data['book_info']

In [31]:
# 매칭 키워드
## 스터디라인
tSL1 = ['기초이론', '기초서', '기본이론', '기본서', '입문이론', '입문서', '개념이론', '개념서', \
       '초보필독서', '기초필독서', '기본필독서', '입문필독서', '개념필독서']

tSL2 = ['OX','서브노트','두문자', '단원별기출문제', '단원별문제', '심화이론', '이론심화', '핵심이론',\
        '유형별문제집', '유형별핵심문제','유형별이론', '유형별기출문제', '유형별문제풀이']

tSL3 = ['0제', '빈출문제', '랜덤문제']

tSL4 = ['취약문제', '취약파트', '심화문제집', '심화문제', '고난이도문제', '고난도문제', '고난도이론',\
        '테마정리', '상위권문제']

tSL5 = ['실전모의고사', '봉투모의고사', '동형모의고사', '더블업', '더블업모의고사', '총정리', '최종',\
        '최종정리', '최종마무리', '필기노트']

## 책유형
title_t = ['입문서', '기초서', '기본서', '이론서', '개념서', '심화서', '요약서', '입문이론', '기초이론',\
           '기본이론', '개념이론', '핵심이론', '합격노트', '필기노트', '요약집']
etc_t = ['입문서', '기초서', '이론서', '개념서', '심화서', '요약서', '입문이론', '기초이론',\
         '기본이론', '개념이론', '핵심이론', '합격노트', '요약집', '이론정리', '기초정리',\
         '기본정리', '핵심정리', '요약정리', '이론요약',  '요약이론', '포인트이론',\
         '개념이해','개념학습']
title_q = ['문제집', '0제', '모의고사', '예상문제', '실전문제', '문제풀이','OX','기출문제','진도별']
etc_q = ['문제집', '0제', '모의고사', '예상문제', '실전문제', '문제풀이', '문제수록',\
           '빈출문제', '연도별기출', '핵심문제', '단원별문제', '심화문제', '심화문제집',\
           '실전문제', '실전모의고사', '동형모의고사', '더블업', '진도별모의고사', '핵심문제',\
           '봉투모의고사', '예상문제', '문제엄선']


## 과목
sub = ['형법', '헌법', '형사법', '경찰행정학', '형사소송법', '경찰학', '경찰실무', '경찰행정법','주관식', '형사소송법', '경찰실무종합', '한국사',\
       '영어', '세법개론', '정보보호론', '범죄학', '회계학', '시스템', '행정법', '행정학', '민법총칙', '상법총칙', '경제학', '통계학', '재정학', '시스템',\
       '네트워크보안', '데이터베이스론', '통신이론', '소프트웨어공학']

## 급수
detail = ['9급','7/8급','7급','8급','6급','5급']


# function

In [11]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result

#1. set_isbn 확인
def search_codes(df, column):
    ls = df[column].unique().tolist()
    ls =np.nan_to_num(ls) # 널 값을 0으로 대체
    ls = [int(l) for l in ls]
    return ls

def search_value_type(data, check_idx):
    for idx in check_idx:
        result = ''
        result_ls = [data.loc[i,'책유형'] for i in idx if type(data.loc[i,'책유형']) != float]
        result_ls = list(set(result_ls))
        print(result_ls)
        if len(result_ls) == 0:
            result = '수작업'
        elif len(result_ls) == 1:
            result = result_ls[0]
        elif len(result_ls) == 2:
            result = '혼합'
        print(result)
        data.loc[idx[0],'책유형'] = result
        print( idx[0], result)
        
def search_value_sl(data, check_idx): ## 스터디라인
    for idx in check_idx:
        result = ''
        result_ls = [data.loc[i,('스터디라인')] for i in idx if type(data.loc[i,('스터디라인')]) != float]
        result_ls = np.unique(np.array(result_ls)).tolist()
        print(result_ls)
        if len(result_ls) == 0:
            result = ['수작업']
        elif len(result_ls) >= 1:
            result = result_ls
        data.loc[idx[0],'스터디라인'] = result
        print( idx[0], result)

# classification

## stdln

In [32]:
## 스터디라인

isbn, title, studyLine, etc, set_isbn = [],[],[],[],[] # 값 저장 리스트

for j in range(len(tit)):
    if code[j] == codes[j]:
        isbn.append(code[j])
        set_isbn.append(codes[j])
        title.append(tit[j])
        studyLine.append(np.NaN)
        etc.append('세트권')
    else:
        if type(tit[j]) != float:
            e = tit[j].replace(' ','')
            if '+' in e or 'all' in e.lower() or 'set' in e.lower()or '세트' in e.lower(): # 혼합일 가능성이 큼
                isbn.append(code[j])
                set_isbn.append(codes[j])
                title.append(tit[j])
                studyLine.append(np.NaN)
                etc.append('수작업')
            else:
                lists = [
                    [1 for i in tSL1 if i in e],
                    [2 for i in tSL2 if i in e],
                    [3 for i in tSL3 if i in e],
                    [4 for i in tSL4 if i in e],
                    [5 for i in tSL5 if i in e]
                ]
    #             max_list = max(lists, key=len)
                result = [set(lst) for lst in lists if lst != []]
                if len(result) != 0:
                    isbn.append(code[j])
                    set_isbn.append(codes[j])
                    title.append(tit[j])
                    studyLine.append(result)
                    etc.append(np.NaN)
                else:
                    isbn.append(code[j])
                    set_isbn.append(codes[j])
                    title.append(tit[j])
                    studyLine.append(np.NaN)
                    etc.append('수작업')
        else:
            print('no tilte')

In [33]:
std = pd.DataFrame({'isbn':isbn, 'set_isbn':set_isbn, '교재명':title,'스터디라인':studyLine,'기타':etc})

In [34]:
column = 'set_isbn' # 컬럼명
# step 1
ls = search_codes(std, column) # setisbn 번호 수집
print(ls)
code = std.loc[:,column] # 전체 isbn 서칭
print(code)
# step 2
check_idx = [np.where(i==code)[0].tolist() for i in ls[1:]]
print(check_idx)
# step 3 # 책유형,또는 스터디라인으로 변경
search_value_sl(std, check_idx)

[9791169660891, 0, 9791169660419, 9791160183627, 9791192105758, 9791165768287, 9791169660846, 9791192405278, 9791160180466]
0     9791169660891
1     9791169660891
2     9791169660891
3     9791169660891
4                 0
5                 0
6                 0
7     9791169660419
8     9791169660419
9     9791169660419
10                0
11                0
12                0
13                0
14                0
15                0
16                0
17                0
18    9791160183627
19    9791160183627
20    9791160183627
21                0
22                0
23                0
24                0
25                0
26                0
27                0
28                0
29                0
30                0
31    9791192105758
32    9791192105758
33    9791192105758
34                0
35                0
36                0
37    9791165768287
38    9791165768287
39    9791165768287
40    9791165768287
41                0
42                0
43              

IndexError: list index out of range

In [35]:
# 데이터 확인
std 

,isbn,set_isbn,교재명,스터디라인,기타
0,9791169660891,9791169660891,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,NaN,세트권
1,9791169660907,9791169660891,2024 선재국어 수비니겨 기본서 1 독해 · 문학,[{1}],NaN
2,9791169660921,9791169660891,2024 선재국어 수비니겨 날로 쓰는 자습서,NaN,수작업
3,9791169660914,9791169660891,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,[{1}],NaN
4,9791169660549,0,2024 이동기 영어 문법·구문 50포인트,NaN,수작업
5,9791169660556,0,2024 이동기 영어 독해 12공식,NaN,수작업
6,9791160183658,0,2024 이동기 공무원 마스터 VOCA 2000+,NaN,수작업
7,9791169660419,9791169660419,2024 문동균 한국사 All in one,NaN,세트권
8,9791169660433,9791169660419,문동균 한국사 all in one 2 - 공무원 한국사,NaN,수작업
9,9791169660426,9791169660419,문동균 한국사 all in one 1 - 공무원 한국사,NaN,수작업


In [37]:
# 작업 완료된 건 대상 
std.to_excel("C://info_data//test_stdln.xlsx", encoding ='utf-8-sig', index = False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# sor

In [ ]:
# 저장 리스트
isbn, title, book_type, etc, set_isbn = [],[],[],[],[]

In [ ]:
for j in range(len(tit)):
    if code[j] == codes[j]:
        isbn.append(code[j])
        set_isbn.append(codes[j])
        title.append(tit[j])
        book_type.append(np.NaN)
        etc.append('세트권')
    else:
        if type(tit[j]) != float:
            e = tit[j].replace(' ','')
            if '+' in e or 'all' in e.lower() or 'set' in e.lower()or '세트' in e.lower(): # 혼합일 가능성이 큼
                isbn.append(code[j])
                set_isbn.append(codes[j])
                title.append(tit[j])
                book_type.append(np.NaN)
                etc.append('수작업')
            else:
                lists = [
                    ['T' for i in title_t if i in e], # 이론서
                    ['W' for i in title_q if i in e] # 문제풀이
                ]
                max_list = max(lists, key=len)
                if len(max_list) != 0:
                    isbn.append(code[j])
                    title.append(tit[j])
                    set_isbn.append(codes[j])
                    book_type.append(max_list[0]) # max_list[0]
                    etc.append('매칭')
                else:
                    isbn.append(code[j])
                    title.append(tit[j])
                    set_isbn.append(codes[j])
                    book_type.append(np.NaN)
                    etc.append('')
        else:
            print('else')

In [ ]:
bookType = pd.DataFrame({'isbn':isbn, 'set_isbn':set_isbn, '교재명':title,'txtb_sor_cd':book_type,'책유형':etc})

In [ ]:
column = 'set_isbn' # 컬럼명
# step 1
ls = search_codes(bookType, column) # setisbn 번호 수집
print(ls)
code = bookType.loc[:,column] # 전체 isbn 서칭
print(code)
# step 2
check_idx = [np.where(i==code)[0].tolist() for i in ls[1:]]
print(check_idx)
# step 3 # 책유형,또는 스터디라인으로 변경
search_value_type(bookType, check_idx)

In [ ]:
# 저장 대상 
sorDF = bookType.loc[bookType['txtb_sor_cd'].isin(['T', 'W']), :]
sorDF.index = range(len(sorDF))
sorDF

In [ ]:
sorDF['txtb_id'] = sorDF['isbn'].apply(lambda x : f'm{x}')
sorDF = sorDF[['txtb_id', 'txtb_sor_cd']]
sorDF.to_excel('C:/police_data/경찰직_책유형_07_10.xlsx',encoding = 'utf-8-sig', index=False)

In [ ]:
handType = bookType.loc[:, ('교재명', 'isbn','set_isbn', 'txtb_sor_cd')]
handType['스터디라인'] = ''
handType

In [ ]:
# 교재 부분 처리 코드
target = book.loc[:,('isbn','set_isbn','txtb_nm')]
target['subject'] = ''
target['studyLine'] = ''
target['bookType'] = ''
target.to_excel("C:/info_data/test_수작업_요청파일091823'.xlsx", encoding = 'utf-8-sig', index = False)